<a href="https://colab.research.google.com/github/akrysm/rnaseq/blob/main/RNA_seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#準備(GoogleDrive, インストール, ディレクトリ・フォルダ設定, コンテンツ確認)

In [ ]:
#@title Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title インストール
import gzip
import locale
# import multiqc
import os
import shutil
import subprocess
import time

# print(multiqc.__version__)


!apt-get -qq install hisat2
!apt-get -qq install samtools
!apt-get -qq install subread
!apt-get -qq install tree
os.environ['PATH'] += ":/usr/local/bin"

print("<<<<<<<<<< hisat2 version >>>>>>>>>>")
!hisat2 --version
print("<<<<<<<<<< samtools version >>>>>>>>>>")
!samtools --version
print("<<<<<<<<<< subread version >>>>>>>>>>")
!subread --version
print("<<<<<<<<<< tree version >>>>>>>>>>")
!tree --version

Selecting previously unselected package tree.
(Reading database ... 121988 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
<<<<<<<<<< hisat2 version >>>>>>>>>>
/usr/bin/hisat2-align-s version 2.2.1
64-bit
Built on Debian
28 September 2021
Compiler: gcc version 11.2.0 (Ubuntu 11.2.0-7ubuntu2) 
Options: -O3   -funroll-loops -g3 -Wdate-time -D_FORTIFY_SOURCE=2 -std=c++11
Sizeof {int, long, long long, void*, size_t, off_t}: {4, 8, 8, 8, 8, 8}
<<<<<<<<<< samtools version >>>>>>>>>>
samtools 1.13
Using htslib 1.13+ds
Copyright (C) 2021 Genome Research Ltd.

Samtools compilation details:
    Features:       build=configure curses=yes 
    CC:             gcc
    CPPFLAGS:       -frelease  -Wdate-time -D_FORTIFY_SOURCE=2
    CFLAGS:         -g -O2 -ffile-prefix-map=�BUILDPATH�=. -flto=auto -ffat-lto-objects -fstack-protector-strong -Wformat -We

In [ ]:
#@title フォルダ作成: MyDrive以下に [RNAseq] を作成
import os

# working directory path
DIR_WORK = "/content/drive/MyDrive" #@param ["/content/drive/MyDrive", "option"] {allow-input: true}

# %cd {DIR_RAWDATA}  # linux
os.chdir(DIR_WORK)
print("現在のディレクトリ:", os.getcwd())

# folder name
directories = ["RNAseq"]

# create folder function
def check_and_create_directory(directory):
    if not os.path.exists(directory):
        print(f"Creating directory: {directory}")
        os.makedirs(directory)
    else:
        print(f"Directory already exists: {directory}. Nothing to do.")

# create folder
for directory in directories:
    check_and_create_directory(directory)

現在のディレクトリ: /content/drive/MyDrive
Directory already exists: RNAseq. Nothing to do.


In [ ]:
#@title フォルダ作成: RNAseq以下に [bamfiles, downloads, fastqc, featureCounts, multiqc, rawdata, trimmed] を作成
import os

# working directory path
DIR_WORK = "/content/drive/MyDrive/RNAseq" #@param ["/content", "/content/drive/MyDrive/RNAseq", "/content/drive/MyDrive/RNAseq/rawdata", "option"] {allow-input: true}

# %cd {DIR_RAWDATA}  # linux
os.chdir(DIR_WORK)
print("現在のディレクトリ:", os.getcwd())

# folder name
directories = ["bamfiles", "downloads", "fastqc", "featureCounts", "multiqc", "rawdata", "trimmed"]

# create folder function
def check_and_create_directory(directory):
    if not os.path.exists(directory):
        print(f"Creating directory: {directory}")
        os.makedirs(directory)
    else:
        print(f"Directory already exists: {directory}. Nothing to do.")

# create folder
for directory in directories:
    check_and_create_directory(directory)

現在のディレクトリ: /content/drive/MyDrive/RNAseq
Directory already exists: bamfiles. Nothing to do.
Directory already exists: downloads. Nothing to do.
Directory already exists: fastqc. Nothing to do.
Directory already exists: featureCounts. Nothing to do.
Directory already exists: multiqc. Nothing to do.
Directory already exists: rawdata. Nothing to do.
Directory already exists: trimmed. Nothing to do.


###シーケンスリードデータ(.fastq.gz)とサンプルリストファイル(sample_list.txt)の準備

/content/drive/MyDrive/RNAseq/rawdata に  


* シーケンスリードデータ(.fastq.gz)
* サンプルリストファイル(sample_list.txt)  


を格納しておく。  
サンプルリストファイルは以下の通り。


| No. | ID  | Info |
|-----|-----|------|
| 1   | ABC | ...  |
| 2   | DEF | ...  |
| 3   | GHI | ...  |
| 4   | JKL | ...  |




In [ ]:
#@title コンテンツ表示
import os

# ワーキングディレクトリ
DIR_WORK = "/content/drive/MyDrive/RNAseq" #@param ["/content", "/content/drive/MyDrive/RNAseq", "/content/drive/MyDrive/RNAseq/rawdata", "option"] {allow-input: true}

# ワーキングディレクトリの移動
os.chdir(DIR_WORK)
print("現在のディレクトリ:", os.getcwd())

# def print_directory_tree(directory, indent=''):
#     print(indent + os.path.basename(directory) + '/')
#     indent += '    '
#     for item in os.listdir(directory):
#         item_path = os.path.join(directory, item)
#         if os.path.isdir(item_path):
#             print_directory_tree(item_path, indent)
#         else:
#             print(indent + item)

# # 現在のワーキングディレクトリのツリー形式を表示
# print_directory_tree(os.getcwd())



## ----------シェルコマンドをGoogle Colaboratoryで実行----------
# !apt-get -qq install tree # tree表示のためのツールをインストール
!tree {DIR_WORK}
# !tree {DIR_RAWDATA}
## -------------------------------------------------------

現在のディレクトリ: /content/drive/MyDrive/RNAseq
/content/drive/MyDrive/RNAseq
├── bamfiles
│   ├── HIP_PC10_sorted.bam
│   ├── HIP_PC10_sorted.bam.bai
│   ├── HIP_PC30_sorted.bam
│   ├── HIP_PC30_sorted.bam.bai
│   ├── HIP_PV_sorted.bam
│   ├── HIP_PV_sorted.bam.bai
│   ├── HIP_SC10_sorted.bam
│   ├── HIP_SC10_sorted.bam.bai
│   ├── HIP_SC30_sorted.bam
│   ├── HIP_SC30_sorted.bam.bai
│   ├── HIP_SV_sorted.bam
│   ├── HIP_SV_sorted.bam.bai
│   ├── NAc_PC10_sorted.bam
│   ├── NAc_PC10_sorted.bam.bai
│   ├── NAc_PC30_sorted.bam
│   ├── NAc_PC30_sorted.bam.bai
│   ├── NAc_PV_sorted.bam
│   ├── NAc_PV_sorted.bam.bai
│   ├── NAc_SC10_sorted.bam
│   ├── NAc_SC10_sorted.bam.bai
│   ├── NAc_SC30_sorted.bam
│   ├── NAc_SC30_sorted.bam.bai
│   ├── NAc_SV_sorted.bam
│   ├── NAc_SV_sorted.bam.bai
│   ├── PFC_PC10_sorted.bam
│   ├── PFC_PC10_sorted.bam.bai
│   ├── PFC_PC30_sorted.bam
│   ├── PFC_PC30_sorted.bam.bai
│   ├── PFC_PV_sorted.bam
│   ├── PFC_PV_sorted.bam.bai
│   ├── PFC_SC10_sorted.bam
│   ├── 

In [ ]:
#@title サンプルリストの確認
import os
import pandas as pd

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
sample_list_file = "sample_list_original.txt"  #@param ["sample_list.txt", "sample_list_original.txt", "option1"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# データフレームに読み込む
df = pd.read_csv(sample_list_file, delimiter='\t')  # デリミターはタブに設定していますが、必要に応じて変更してください

# 最初の数行を表示
# print(df.head())
display(df.head().style)

# with open(sample_list_file, 'r') as file:
#     for i in range(5):  # 最初の5行を表示
#         line = file.readline()
#         print(line)

,No,SampleID,Analyte,Group,Drug,Drug1,Drug2
0,1,HIP_PC10,HIP,PC10,PCP_CLZ10,PCP,CLZ10
1,2,HIP_PC30,HIP,PC30,PCP_CLZ30,PCP,CLZ30
2,3,HIP_PV,HIP,PV,PCP_VEH,PCP,VEH
3,4,HIP_SC10,HIP,SC10,SAL_CLZ10,SAL,CLZ10
4,5,HIP_SC30,HIP,SC30,SAL_CLZ30,SAL,CLZ30


#トリミング(Trimmomatic)

In [ ]:
#@title ダウンロードトリミングツール: Trimmomatic
import os
import subprocess

# ディレクトリ
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
DIR_DOWNLOADS = "../downloads"  #@param ["../downloads", "/content/drive/MyDrive/RNAseq/downloads", "option1"] {allow-input: true}
DIR_TRIMMOMATIC = os.path.join(DIR_DOWNLOADS, "Trimmomatic-0.39")

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# すでにTrimmomaticが存在するかどうかをチェック
if not os.path.exists(DIR_TRIMMOMATIC):
    # TrimmomaticのZIPファイルをダウンロード
    trimmomatic_zip_url = "http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.39.zip"  #@param ["http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.39.zip", "option1", "option2"] {allow-input: true}
    trimmomatic_zip_file = os.path.basename(trimmomatic_zip_url)
    subprocess.run(["wget", trimmomatic_zip_url], cwd=DIR_DOWNLOADS)

    # TrimmomaticのZIPファイルを展開
    subprocess.run(["unzip", trimmomatic_zip_file], cwd=DIR_DOWNLOADS)
else:
    print("Trimmomaticはすでに展開されています。")


## ----------シェルコマンドをPythonコードで実行----------
# Trimmomaticをダウンロードして展開する
# %cd /content/drive/MyDrive/RNAseq/downloads
# !wget http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.39.zip
# !unzip Trimmomatic-0.39.zip
## -------------------------------------------------------


Trimmomaticはすでに展開されています。


In [ ]:
#@title トリミング（未実施サンプルデータのみ処理）: Trimmomatic
import os
import subprocess
import time

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
sample_list_file = "sample_list.txt"  #@param ["sample_list.txt", "sample_list_original.txt", "option1"] {allow-input: true}
java_file = "../downloads/Trimmomatic-0.39/trimmomatic-0.39.jar"  #@param ["../downloads/Trimmomatic-0.39/adapters/TruSeq2-PE.fa:2:30:10", "option1", "option2"] {allow-input: true}
illuminaclip_file = "../downloads/Trimmomatic-0.39/adapters/TruSeq2-PE.fa:2:30:10"  #@param ["../downloads/Trimmomatic-0.39/adapters/TruSeq2-PE.fa:2:30:10", "option1", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# サンプルリストをファイルから読み取る
with open(sample_list_file, "r") as f:
    sample_list = f.read().splitlines()

# サンプルリストをループで処理
for sample_line in sample_list[1:]:  # 1行目をスキップして残りの行を処理
    # サンプル名を分割して取得
    sample_data = sample_line.strip().split("\t")
    sample_number = sample_data[0]  # サンプル番号

    # 出力ファイル名を検査してトリミング済みかどうかを確認
    output_forward_paired = f"../trimmed/paired_{sample_data[1]}_1.trim.fastq.gz"
    output_reverse_paired = f"../trimmed/paired_{sample_data[1]}_2.trim.fastq.gz"
    if os.path.exists(output_forward_paired) and os.path.exists(output_reverse_paired):
        print(f"Sample No. {sample_number}, <{sample_data[1]}> has already been trimmed. Skipping...")
        continue

    # 入力ファイルのフルパスを指定
    input_forward = f"{sample_data[1]}_1.fastq.gz"
    input_reverse = f"{sample_data[1]}_2.fastq.gz"
    output_forward_paired = f"../trimmed/paired_{sample_data[1]}_1.trim.fastq.gz"
    output_forward_unpaired = f"../trimmed/unpaired_{sample_data[1]}_1.trim.fastq.gz"
    output_reverse_paired = f"../trimmed/paired_{sample_data[1]}_2.trim.fastq.gz"
    output_reverse_unpaired = f"../trimmed/unpaired_{sample_data[1]}_2.trim.fastq.gz"

    # ファイル名を出力して確認
    print(f"Sample No. {sample_number}, Forward Input File: {input_forward}, Reverse Input File: {input_reverse}")
    print(f"Sample No. {sample_number}, Forward Output File: {output_forward_paired}, {output_forward_unpaired}, Reverse Output File: {output_reverse_paired}, {output_reverse_unpaired}")

    # 処理開始時間を記録
    start_time = time.time()
    print(f"Processing started for sample {sample_data[1]} at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

    # コマンドを構築
    trimmomatic_cmd = [
        "java", "-jar", java_file, "PE", "-phred33",
        input_forward, input_reverse,
        output_forward_paired, output_forward_unpaired,
        output_reverse_paired, output_reverse_unpaired,
        f"ILLUMINACLIP:{illuminaclip_file}"
    ]

    # コマンド実行前のメッセージを表示
    print(f"Starting trimming for sample {sample_data[1]}...")

    # trimmomaticコマンドを実行
    result = subprocess.run(trimmomatic_cmd, capture_output=True, text=True)

    # 結果を表示
    print(f"Trimming for sample {sample_data[1]} finished with return code: {result.returncode}")
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print(result.stderr)

    # 処理終了時間を記録
    end_time = time.time()

    # 処理時間を計算して表示
    processing_time = end_time - start_time
    print(f"Processing time for sample {sample_data[1]}: {processing_time} seconds")

    # 処理の区切りを示すメッセージを表示
    print("------------------------------------------------------")


Sample No. 1, <HIP_PC10> has already been trimmed. Skipping...
Sample No. 2, <HIP_PC30> has already been trimmed. Skipping...
Sample No. 3, <HIP_PV> has already been trimmed. Skipping...
Sample No. 4, <HIP_SC10> has already been trimmed. Skipping...
Sample No. 5, <HIP_SC30> has already been trimmed. Skipping...
Sample No. 6, <HIP_SV> has already been trimmed. Skipping...
Sample No. 7, <NAc_PC10> has already been trimmed. Skipping...
Sample No. 8, <NAc_PC30> has already been trimmed. Skipping...
Sample No. 9, <NAc_PV> has already been trimmed. Skipping...
Sample No. 10, <NAc_SC10> has already been trimmed. Skipping...
Sample No. 11, <NAc_SC30> has already been trimmed. Skipping...
Sample No. 12, <NAc_SV> has already been trimmed. Skipping...
Sample No. 13, <PFC_PC10> has already been trimmed. Skipping...
Sample No. 14, <PFC_PC30> has already been trimmed. Skipping...
Sample No. 15, <PFC_PV> has already been trimmed. Skipping...
Sample No. 16, <PFC_SC10> has already been trimmed. Skippin

#マッピング(HISAT2)

In [ ]:
#@title ゲノム配列情報（FNA）のダウンロード: HISAT2
import gzip
import os
import shutil
import subprocess

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
DIR_DOWNLOADS = "../downloads"  #@param ["../downloads", "/content/drive/MyDrive/RNAseq/downloads", "option1"] {allow-input: true}
fna_file = "GCF_000001635.27_GRCm39_genomic.fna"  #@param ["GCF_000001635.27_GRCm39_genomic.fna.gz", "option1", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# ファイルが存在するかチェック
if not os.path.exists(os.path.join(DIR_DOWNLOADS, fna_file)):
    # ダウンロード
    fna_file_url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.27_GRCm39/GCF_000001635.27_GRCm39_genomic.fna.gz"  #@param ["https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.27_GRCm39/GCF_000001635.27_GRCm39_genomic.fna.gz", "option1", "option2"] {allow-input: true}
    subprocess.run(["wget", fna_file_url, "-P", DIR_DOWNLOADS])

    # 解凍
    fna_file_path = os.path.join(DIR_DOWNLOADS, fna_file + ".gz")
    output_file_path = os.path.join(DIR_DOWNLOADS, fna_file)
    with gzip.open(fna_file_path, 'rb') as f_in:
        with open(output_file_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
else:
    print("ファイルがすでに存在します。")


ファイルがすでに存在します。


In [ ]:
#@title （インストール: HISAT2）
# import os
# !apt-get -qq install hisat2
# !apt-get -qq install samtools
# os.environ['PATH'] += ":/usr/local/bin"
# !hisat2 --version
# !samtools --version

In [ ]:
#@title インデックスの作成: HISAT2
import os
import subprocess
import time

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
genomic_fna = "../downloads/GCF_000001635.27_GRCm39_genomic.fna"  #@param ["../downloads/GCF_000001635.27_GRCm39_genomic.fna", "option1", "option2"] {allow-input: true}
index_path = "../downloads/GCF_000001635.27_GRCm39_genomic_index"  #@param ["../downloads/GCF_000001635.27_GRCm39_genomic_index", "option1", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# 処理開始時間を記録
start_time = time.time()
print(f"Processing started at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

# インデックスファイルが存在するかチェック
if not os.path.exists(index_path + ".1.ht2"):
    # インデックスを作成
    print("Creating index...")
    subprocess.run(["hisat2-build", genomic_fna, index_path])
    print("Index creation completed.")
else:
    print("インデックスファイルはすでに存在します。")

# 処理終了時間を記録
end_time = time.time()

# 処理時間を計算して表示
processing_time = end_time - start_time
print(f"Processing time: {processing_time} seconds.")

# 2h 5m 2s


Creating index...
Index creation completed.


In [ ]:
#@title マッピング（未実施サンプルデータのみ処理）: HISAT2
import locale
import os
import subprocess
import time

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
sample_list_file = "sample_list_original.txt"  #@param ["sample_list.txt", "sample_list_original.txt", "option1"] {allow-input: true}
index_path = "../downloads/GCF_000001635.27_GRCm39_genomic_index"  #@param ["../downloads/GCF_000001635.27_GRCm39_genomic_index", "option1", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# サンプルリストファイルからサンプルを取得し、HISAT2を実行
with open(sample_list_file, "r") as f:
    for line in f.readlines()[1:]:
        sample = line.strip().split()[1]
        print(f"Processing sample: {sample}...")
        file1 = f"../trimmed/paired_{sample}_1.trim.fastq.gz"
        file2 = f"../trimmed/paired_{sample}_2.trim.fastq.gz"
        bam_file = f"../bamfiles/{sample}_sorted.bam"

        # 処理開始時間を記録
        start_time = time.time()
        print(f"Processing started for sample {sample} at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

        # すでに処理されたサンプルかどうかを確認
        if os.path.exists(bam_file):
            print(f"Sample {sample} has already been processed. Skipping...")
            continue

        # HISAT2を実行
        hisat2_cmd = [
            "hisat2", "-p", "1", "-x", index_path, "-1", file1, "-2", file2
        ]
        hisat2_proc = subprocess.Popen(hisat2_cmd, stdout=subprocess.PIPE)

        # samtoolsを使ってバムファイルに変換
        samtools_cmd = [
            "samtools", "sort", "-@ 1", "-", "-o", bam_file
        ]
        samtools_proc = subprocess.Popen(samtools_cmd, stdin=hisat2_proc.stdout)
        samtools_proc.communicate()

        # バムファイルにインデックスを作成
        subprocess.run(["samtools", "index", "-@", "1", bam_file])

        # 処理終了時間を記録
        end_time = time.time()

        # 処理時間を計算して表示
        processing_time = end_time - start_time
        print(f"Sample {sample} processing complete. Processing time: {processing_time} seconds.")


#カウント(featureCount(subread))

In [ ]:
#@title アノテーション（GTF）のダウンロード: featureCount(subread)
import gzip
import os
import shutil
import subprocess

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
DIR_DOWNLOADS = "../downloads"  #@param ["../download", "/content/drive/MyDrive/RNAseq/downloads", "option1"] {allow-input: true}
gtf_file = "GCF_000001635.27_GRCm39_genomic.gtf"  #@param ["GCF_000001635.27_GRCm39_genomic.gtf", "option1", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# ファイルが存在するかチェック
if not os.path.exists(os.path.join(DIR_DOWNLOADS, gtf_file)):
    # ダウンロード
    gtf_file_url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.27_GRCm39/GCF_000001635.27_GRCm39_genomic.gtf.gz"  #@param ["https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.27_GRCm39/GCF_000001635.27_GRCm39_genomic.gtf.gz", "option1", "option2"] {allow-input: true}
    subprocess.run(["wget", gtf_file_url, "-P", DIR_DOWNLOADS])

    # 解凍
    gtf_file_path = os.path.join(DIR_DOWNLOADS, gtf_file + ".gz")
    output_file_path = os.path.join(DIR_DOWNLOADS, gtf_file)
    with gzip.open(gtf_file_path, 'rb') as f_in:
        with open(output_file_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
else:
    print("ファイルがすでに存在します。")

ファイルがすでに存在します。


In [ ]:
#@title リードカウント: featureCount(subread)
import os
import subprocess
import time
!apt-get -qq install subread
os.environ['PATH'] += ":/usr/local/bin"

# ワーキングディレクトリ
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
os.chdir(DIR_WORK)

# ファイルパス
annotation_file = "/content/drive/MyDrive/RNAseq/downloads/GCF_000001635.27_GRCm39_genomic.gtf"  #@param ["/content/drive/MyDrive/RNAseq/downloads/GCF_000001635.27_GRCm39_genomic.gtf", "../downloads/GCF_000001635.27_GRCm39_genomic.gtf", "option1", "option2"] {allow-input: true}
input_file = "/content/drive/MyDrive/RNAseq/bamfiles/*_sorted.bam" #@param ["/content/drive/MyDrive/RNAseq/bamfiles/*_sorted.bam", "../bamfiles/*_sorted.bam", "option1", "option2"] {allow-input: true}
output_file = "/content/drive/MyDrive/RNAseq/featureCounts/featureCounts.txt"  #@param ["/content/drive/MyDrive/RNAseq/featureCounts/featureCounts.txt", "../featureCounts/featureCounts.txt", "option1", "option2"] {allow-input: true}

# featureCountsの定義
# featureCounts_cmd = [
#     "featureCounts",
#     "-F", "GTF",
#     "-t", "CDS",
#     "-g", "gene_id",
#     "-a", annotation_file,
#     "-o", output_file,
#     "-p", input_file
# ]

# 実行時間を記録
start_time = time.time()
print(f"Processing started at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

# featureCountsを実行
# subprocess.run(featureCounts_cmd)
!featureCounts -F GTF -t CDS -g gene_id \
    -a $annotation_file \
    -o $output_file \
    -p $input_file

# 実行時間を計算して表示
end_time = time.time()
execution_time = end_time - start_time
print(f"Processing time: {execution_time} seconds.")


#その他（テストなど）

### QC

In [ ]:
!pip install biopython

In [ ]:
from Bio import SeqIO
import glob
import gzip
import os
import time

# ワーキングディレクトリ
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata" #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
os.chdir(DIR_WORK)

# ファイルパス
input_file_1 = "/content/drive/MyDrive/RNAseq/rawdata/*.fastq.gz" #@param ["/*.fastq.gz", "../trimmed/*.fastq.gz", "../*/*.fastq.gz", "", "option"] {allow-input: true}
input_file_2 = "../trimmed/*.fastq.gz" #@param [""./*.fastq.gz", "../trimmed/*.fastq.gz", "../*/*.fastq.gz", "", "option"] {allow-input: true}
input_file_3 = "" #@param ["./*.fastq.gz", "../trimmed/*.fastq.gz", "../*/*.fastq.gz", "", "option"] {allow-input: true}
input_patterns = [input_file_1, input_file_2, input_file_3]
# input_patterns = ["/content/drive/MyDrive/RNAseq/rawdata/*.fastq.gz",
#                   "/content/drive/MyDrive/RNAseq/trimmed/*.fastq.gz",
#                   "/content/drive/MyDrive/RNAseq/other/*.fastq.gz"]
output_dir = "/content/drive/MyDrive/RNAseq/fastqc" #@param ["/content/drive/MyDrive/RNAseq/fastqc", "option1", "option2"] {allow-input: true}

# Fastqファイルのリストを取得
fastq_files = []
for pattern in input_patterns:
    fastq_files.extend(glob.glob(pattern))

def generate_qc_report(fastq_files, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for fastq_file in fastq_files:
        output_file = os.path.join(output_dir, f"{os.path.basename(fastq_file)}.qc_report.txt")
        with open(output_file, "w") as report:
            report.write("File\tTotal Sequences\tAverage Quality\n")
            total_sequences = 0
            total_quality = 0
            with gzip.open(fastq_file, "rt") as handle:
                for record in SeqIO.parse(handle, "fastq"):
                    total_sequences += 1
                    total_quality += sum(record.letter_annotations["phred_quality"])
            average_quality = total_quality / total_sequences if total_sequences != 0 else 0
            report.write(f"{fastq_file}\t{total_sequences}\t{average_quality:.2f}\n")

# 実行時間を記録
start_time = time.time()
print(f"Processing started at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

# QCレポートの生成
generate_qc_report(fastq_files, output_dir)

# 実行時間を計算して表示
end_time = time.time()
execution_time = end_time - start_time
print(f"Processing time: {execution_time} seconds.")

In [ ]:
from Bio import SeqIO
import glob
import gzip
import os
import time

# ワーキングディレクトリ
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata" #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
os.chdir(DIR_WORK)

# ファイルパス
input_file_1 = "/content/drive/MyDrive/RNAseq/rawdata/*.fastq.gz" #@param ["/*.fastq.gz", "../trimmed/*.fastq.gz", "../*/*.fastq.gz", "", "option"] {allow-input: true}
input_file_2 = "../trimmed/*.fastq.gz" #@param [""./*.fastq.gz", "../trimmed/*.fastq.gz", "../*/*.fastq.gz", "", "option"] {allow-input: true}
input_file_3 = "" #@param ["./*.fastq.gz", "../trimmed/*.fastq.gz", "../*/*.fastq.gz", "", "option"] {allow-input: true}
input_patterns = [input_file_1, input_file_2, input_file_3]
output_dir = "/content/drive/MyDrive/RNAseq/fastqc"

# 処理済みファイルを記録するセット
processed_files = set()

def generate_qc_report(fastq_files, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for fastq_file in fastq_files:
        # すでに処理済みの場合はスキップ
        if fastq_file in processed_files:
            print(f"Skipping {fastq_file}: Already processed.")
            continue

        # 処理済みファイルに追加
        processed_files.add(fastq_file)

        output_file = os.path.join(output_dir, f"{os.path.basename(fastq_file)}.qc_report.html")
        with open(output_file, "w") as report:
            report.write("<html><body>")
            report.write("<h1>QC Report</h1>")
            report.write("<table>")
            report.write("<tr><th>File</th><th>Total Sequences</th><th>Average Quality</th></tr>")
            total_sequences = 0
            total_quality = 0
            with gzip.open(fastq_file, "rt") as handle:
                for record in SeqIO.parse(handle, "fastq"):
                    total_sequences += 1
                    total_quality += sum(record.letter_annotations["phred_quality"])
                average_quality = total_quality / total_sequences if total_sequences != 0 else 0
                report.write(f"<tr><td>{fastq_file}</td><td>{total_sequences}</td><td>{average_quality:.2f}</td></tr>")
            report.write("</table>")
            report.write("</body></html>")
        print(f"Generated QC report for {fastq_file}.")

# 実行時間を記録
start_time = time.time()
print(f"Processing started at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

# Fastqファイルのリストを取得
fastq_files = []
for pattern in input_patterns:
    fastq_files.extend(glob.glob(pattern))

# QCレポートの生成
generate_qc_report(fastq_files, output_dir)

# 実行時間を計算して表示
end_time = time.time()
execution_time = end_time - start_time
print(f"Processing time: {execution_time} seconds.")


###(クオリティコントロールレポート(MultiQC): リポートが出力されない)

In [ ]:
import os

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content"  #@param ["/content", "/content/drive/MyDrive/RNAseq/rawdata", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

!git clone https://github.com/MultiQC/MultiQC_Notebook.git



In [ ]:
# %pip install --force-reinstall --upgrade git+https://github.com/ewels/MultiQC.git

In [ ]:
import multiqc
print(multiqc.__version__)

1.22.dev0


In [ ]:
# (クオリティコントロールレポート: MultiQC): リポートが出力されない
# %pip install --force-reinstall --upgrade git+https://github.com/ewels/MultiQC.git

import os
import subprocess
import time
import multiqc
print(multiqc.__version__)

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"  #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}
DIR_MULTIQC = "/content/drive/MyDrive/RNAseq/multiqc"  #@param ["../multiqc", "/content/drive/MyDrive/RNAseq/multiqc", "option1"] {allow-input: true}

# 入力ディレクトリのリスト
input_path_1 = "/content/drive/MyDrive/RNAseq/fastqc"  #@param ["/content/drive/MyDrive/RNAseq/fastqc", "/content/drive/MyDrive/RNAseq/rawdata", "", "option1"] {allow-input: true}
input_path_2 = ""  #@param ["../trimmed", "/content/drive/MyDrive/RNAseq/trimmed", "", "option1"] {allow-input: true}
input_path_3 = ""  #@param ["", "option1", "option2"] {allow-input: true}

DIR_INPUT = [input_path_1, input_path_2, input_path_3]
DIR_ANALYSIS = DIR_INPUT

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

# 処理開始時間を記録
start_time = time.time()
print(f"Processing started at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

# MultiQCを実行
multiqc.run(
            # input_dir= DIR_INPUT,
            output_dir=DIR_MULTIQC,
            analysis_dir=DIR_ANALYSIS)

print("MultiQC processing completed.")

# 処理終了時間を記録
end_time = time.time()

# 処理時間を計算して表示
processing_time = end_time - start_time
print(f"Processing time: {processing_time} seconds.")


In [ ]:
import IPython

# Best if using Google Colab
IPython.display.HTML(filename='./multiqc_report.html')

# Best if running locally
# IPython.display.IFrame('./multiqc_report.html', '100%', 600)

###(seqkit: 実装できていない)

In [ ]:
#@title (seqkit: 実装できていない)
# %cd /content

# !git clone https://github.com/shenwei356/seqkit.git
# %cd seqkit
# !make
# !sudo make install

In [ ]:
## ----------シェルコマンドをPythonコードで実行----------
# Trimmomaticをダウンロードして展開する
# %cd /content/drive/MyDrive/RNAseq/downloads
# !wget http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.39.zip
# !unzip Trimmomatic-0.39.zip
## -------------------------------------------------------

###(トリミング: 実行実績あり: Trimmomatic)

In [ ]:
#(トリミング: 実行実績あり: Trimmomatic)

import os
import subprocess
import time

# ファイルパス
%cd /content/drive/MyDrive/RNAseq/rawdata
sample_list_file = "sample_list.txt"  #@param ["sample_list.txt", "sample_list_original.txt", "option1"] {allow-input: true}
ILLUMINACLIP_file = "../downloads/Trimmomatic-0.39/adapters/TruSeq2-PE.fa:2:30:10"  #@param ["../downloads/Trimmomatic-0.39/adapters/TruSeq2-PE.fa:2:30:10", "option1", "option2"] {allow-input: true}
JAVA_file = "../downloads/Trimmomatic-0.39/trimmomatic-0.39.jar"  #@param ["../downloads/Trimmomatic-0.39/trimmomatic-0.39.jar", "option1", "option2"] {allow-input: true}

# サンプルリストをファイルから読み取る
with open(sample_list_file, "r") as f:
    sample_list = f.read().splitlines()

# サンプルリストをループで処理
for sample_line in sample_list[1:]:  # 1行目をスキップして残りの行を処理
    # サンプル名を分割して取得
    sample_data = sample_line.strip().split("\t")
    sample = sample_data[0]  # サンプル名
    # 入力ファイルのフルパスを指定
    input_forward = f"{sample_data[1]}_1.fastq.gz"
    input_reverse = f"{sample_data[1]}_2.fastq.gz"
    output_forward = f"../trimmed/paired_{sample_data[1]}_1.trim.fastq.gz ../trimmed/unpaired_{sample_data[1]}_1.trim.fastq.gz "
    output_reverse = f"../trimmed/paired_{sample_data[1]}_2.trim.fastq.gz ../trimmed/unpaired_{sample_data[1]}_2.trim.fastq.gz "

    # ファイル名を出力して確認
    print(f"Sample: {sample}, Forward Input File: {input_forward}, Reverse Input File: {input_reverse}")
    print(f"Sample: {sample}, Forward Output File: {output_forward}, Reverse Output File: {output_reverse}")

    # 処理開始時間を記録
    start_time = time.time()

    # コマンドを構築
    trimmomatic_cmd = f"java -jar {JAVA_file} PE -phred33 {input_forward} {input_reverse} " \
                          f"{output_forward} {output_reverse} " \
                          f"ILLUMINACLIP:{ILLUMINACLIP_file}"

    # コマンド実行前のメッセージを表示
    print(f"Starting trimming for sample {sample_data[1]}...")

    # trimmomaticコマンドを実行
    result = subprocess.run(trimmomatic_cmd, shell=True, capture_output=True, text=True)

    # 結果を表示
    print(f"Trimming for sample {sample_data[1]} finished with return code: {result.returncode}")
    print(result.stdout)
    print(result.stderr)

    # 処理終了時間を記録
    end_time = time.time()

    # 処理時間を計算して表示
    processing_time = end_time - start_time
    print(f"Processing time for sample {sample_data[1]}: {processing_time} seconds")

    # 処理の区切りを示すメッセージを表示
    print(f"------------------------------------------------------")


###(フォルダとファイルの削除→必要があればアクティブにして実行)

In [ ]:
# import os
# import shutil

# 削除するディレクトリの親ディレクトリのパス
# parent_directory = "/content/drive/MyDrive/RNAseq" #@param ["/content/drive/MyDrive/RNAseq/rawdata", "option1", "option2"] {allow-input: true}

# 削除するサブディレクトリのリスト
# subdirectories = ["downloads", "trimmed", "bamfiles", "featureCounts", "fastp", "fastp.1", "seqkit"]

# サブディレクトリとファイルを削除
# for subdir in subdirectories:
#     path = os.path.join(parent_directory, subdir)
#     if os.path.isdir(path):
#         os.rmdir(path)
#         print(f"{path} を削除しました。")
#     elif os.path.isfile(path):
#         os.remove(path)
#         print(f"{path} を削除しました。")
#     else:
#         print(f"{path} は存在しません。")

# サブディレクトリを再帰的に削除
# for subdir in subdirectories:
#     path = os.path.join(parent_directory, subdir)
#     if os.path.exists(path):
#         if os.path.isdir(path):
#             shutil.rmtree(path)
#             print(f"{path} を再帰的に削除しました。")
#         else:
#             print(f"{path} はディレクトリではないため削除できません。")
#     else:
#         print(f"{path} は存在しません。")

/content/drive/MyDrive/RNAseq/downloads は存在しません。
/content/drive/MyDrive/RNAseq/trimmed は存在しません。
/content/drive/MyDrive/RNAseq/bamfiles は存在しません。
/content/drive/MyDrive/RNAseq/featureCounts は存在しません。
/content/drive/MyDrive/RNAseq/fastp は存在しません。
/content/drive/MyDrive/RNAseq/fastp.1 は存在しません。
/content/drive/MyDrive/RNAseq/seqkit を再帰的に削除しました。


### miniconda

In [ ]:
import os

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content"  #@param ["/content", "/content/drive/MyDrive/RNAseq/rawdata", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
os.chdir(DIR_WORK)

In [ ]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2024-03-26 11:09:29--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.16.246.55, 104.16.245.55, 2606:4700::6810:f637, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.16.246.55|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2024-03-26 11:09:29--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 50.1M 1s
    50K .......... .......... .......... .......... ..........  0%

In [ ]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes


Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cryptography-35.0.0        |   py36hd23ed53_0         1.5 MB
    setuptools-58.0.4          |   py36h06a4308_0         979 KB
    openssl-1.1.1w             |       h7f8727e_0         3.8 MB
    conda-package-handling-1.7.3|   py36h27cfd23_1         946 KB
    libgcc-ng-9.1.0            |       hdf63c60_0         8.1 MB
    requests-2.27.1            |     pyhd3eb1b0_0          52 KB
    charset-normalizer-2.0.4   |     pyhd3eb1b0_0          33 KB
    tqdm-4.63.0                |     pyhd3eb1b0_0          80 KB
    readline-8.1.2             |       h7f8727e_1         423 KB
    tk-8.6.12                  |       h1ccaba5_0         3.3 MB
    yaml-0.2.5                 |       h7b6447c_0  

cryptography-35.0.0  |  1.5 MB | ########## | 100% 
setuptools-58.0.4    |  979 KB | ########## | 100% 
openssl-1.1.1w       |  3.8 MB | ########## | 100% 
conda-package-handli |  946 KB | ########## | 100% 
libgcc-ng-9.1.0      |  8.1 MB | ########## | 100% 
requests-2.27.1      |   52 KB | ########## | 100% 
charset-normalizer-2 |   33 KB | ########## | 100% 
tqdm-4.63.0          |   80 KB | ########## | 100% 
readline-8.1.2       |  423 KB | ########## | 100% 
tk-8.6.12            |  3.3 MB | ########## | 100% 
yaml-0.2.5           |   87 KB | ########## | 100% 
brotlipy-0.7.0       |  349 KB | ########## | 100% 
urllib3-1.26.8       |  100 KB | ########## | 100% 
xz-5.2.5             |  389 KB | ########## | 100% 
cffi-1.14.6          |  224 KB | ########## | 100% 
conda-4.10.3         |  3.1 MB | ########## | 100% 
libstdcxx-ng-9.1.0   |  4.0 MB | ########## | 100% 
_libgcc_mutex-0.1    |    3 KB | ########## | 100% 
pycparser-2.21       |   94 KB | ########## | 100% 
sqlite-3.38.

In [ ]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.6/site-packages"))


In [ ]:
%%bash
conda install -c bioconda fastqc multiqc


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed


                                                                                    
UnsatisfiableError: The following specifications were found to be incompatible with each other:

Output in format: Requested package -> Available versions

Package libstdcxx-ng conflicts for:
multiqc -> matplotlib-base[version='>=2.1.1'] -> libstdcxx-ng[version='>=11.2.0|>=7.5.0|>=9.3.0|>=7.3.0|>=7.2.0']
fastqc -> openjdk[version='>=8.0.144'] -> libstdcxx-ng[version='>=7.5.0']
python=3.6 -> libstdcxx-ng[version='>=7.2.0|>=7.3.0|>=7.5.0']

Package zlib conflicts for:
python=3.6 -> zlib[version='>=1.2.11,<1.3.0a0']
python=3.6 -> sqlite[version='>=3.35.4,<4.0a0'] -> zlib[version='>=1.2.12,<1.3.0a0|>=1.2.13,<1.3.0a0|>=1.2.13,<2.0a0']

Package perl conflicts for:
multiqc -> git -> perl=5
fastqc -> perl=5.22.0
fastqc -> perl-threaded -> perl[version='>=5.22.0,<5.23.0']

Package libgcc-ng conflicts for:
python=3.6 -> libgcc-ng[version='>=7.2.0|>=7.3.0|>=7.5.0']
python=3.6 -> ncurses[version='>=6.2,<7.0a0'] ->

CalledProcessError: Command 'b'conda install -c bioconda fastqc multiqc \n'' returned non-zero exit status 1.

In [ ]:
%%bash
fastqc  --version

/bin/bash: line 1: fastqc: command not found


In [ ]:
from Bio import SeqIO
import glob
import gzip
import os
import subprocess
import time

# ワーキングディレクトリ
DIR_WORK = "/content/drive/MyDrive/RNAseq/rawdata"
os.chdir(DIR_WORK)

# ファイルパス
input_file_1 = "/content/drive/MyDrive/RNAseq/rawdata/*.fastq.gz" #@param ["/*.fastq.gz", "../trimmed/*.fastq.gz", "../*/*.fastq.gz", "", "option"] {allow-input: true}
input_file_2 = "../trimmed/*.fastq.gz" #@param [""./*.fastq.gz", "../trimmed/*.fastq.gz", "../*/*.fastq.gz", "", "option"] {allow-input: true}
input_file_3 = "" #@param ["./*.fastq.gz", "../trimmed/*.fastq.gz", "../*/*.fastq.gz", "", "option"] {allow-input: true}
input_patterns = [input_file_1, input_file_2, input_file_3]
output_dir = "/content/drive/MyDrive/RNAseq/fastqc" #@param ["/content/drive/MyDrive/RNAseq/fastqc", "", "option"] {allow-input: true}

# 実行時間を記録
start_time = time.time()
print(f"Processing started at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

# 実行コマンド
for pattern in input_patterns:
    # パターンにマッチするファイルのリストを取得
    files = glob.glob(pattern)
    for file in files:
        # QCレポートの出力ファイル名
        qc_report_file = os.path.join(output_dir, f"{os.path.basename(file)}.html")
        # 出力ファイルが存在するか確認
        if os.path.isfile(qc_report_file):
            print(f"QC report for {file} already exists. Skipping.")
        else:
            # fastqc実行
            subprocess.run(["fastqc", "--outdir", output_dir, file])

# 実行時間を計算して表示
end_time = time.time()
execution_time = end_time - start_time
print(f"Processing time: {execution_time} seconds.")

Processing started at 2024-03-26 11:23:13


FileNotFoundError: [Errno 2] No such file or directory: 'fastqc'